In [4]:
# uses Python 3.8.5

<font face=Nunito size=5> Code </font>

How to Build a Profitable Python Trading Algorithm in 5 Minutes

**<u>Model Components</u>**  

1. IWO - Russeel 2000 Growth ETF  
1. TLT - Treasury Bond ETF  
1. QQQ - NASDAQ ETF  

Source: StockCharts or Yahoo Finance

**<u>Model Rules</u>**  
- Buy QQQ when IWO is GREATER than it's 150 EMA (or) IWO's 5 EMA is GREATED than it's 10 EMA.
- Otherwise buy TLT

In [6]:
# imports
import pandas as pf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# part 1: get data